In [6]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = ' + '"' + IPython.notebook.notebook_name + '"')

In [2]:
def WaitIfNecessary(Processes,MaxOpenProcesses,Frequency = 2):
    Poll = [p['Process'].poll() for p in Processes if p['Process'].poll() is None]
    while len(Poll)>=MaxOpenProcesses:
        time.sleep(Frequency)
        Poll = [p['Process'].poll() for p in Processes if p['Process'].poll() is None]
    for p in Processes:
        status = p['Process'].poll()
        p['LastStatus'] = status
        if status is not None and not p['Printed']:
            print('HSIP ID: {}: {}'.format(p['HSIP_ID'],status))
            p['Printed'] = True
    return(Processes)

In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 10
Project_DIR = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO12\HSIP_TrackingTool\HSIPTrackingGIS'
Project_GDB = Project_DIR + '\\HSIPGISFiles.gdb'
AnalysisDir = os.path.join(Project_DIR,'AnalysisFiles')
AnalysisDir2 = os.path.join(Project_DIR,'AnalysisFiles2')

# IRIS Dataset
IRISPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\5.0_Roadway_Layer\IRIS'
IRIS_Route = {y:os.path.join(IRISPath,'HWY'+str(y)+'_route.shp') for y in Years}
IRIS_Tab   = {y:os.path.join(IRISPath,'HWY'+str(y)+'_table.dbf') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]

IRISPath_tab = r'C:\Users\mr068144\Downloads\Curvature'
IRIS_Tab   = {y:os.path.join(IRISPath_tab,'Curves_' + str(y) + '.mdb\\HWY'+str(y)+'_table_HCurve') for y in Years}
IRIS_Tab[2012] = IRIS_Tab[2013]


# Intersections Dataset
#Intersections = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\7_BaseData\Master_BaseFiles_Data\6.0_Intersection_Layer\Intersection2014.mdb\Intersection2014_364596_CleanedUp'
IntPG = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO13\2017LocalFivePercent\WorkData\Intersection\Step_1_Intersection_PeerGroupAssignment\Intersection2014.mdb\Intersection2014_364722_CleanedUp'
Intersections = {year:IntPG for year in Years}
IntPG = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO12\HSIP_TrackingTool\1.0_SourceData'
IntPG = r'C:\Users\mr068144\Downloads\6.0_Intersection_Layer\IRIS_Ints.gdb\Int2014_PG'
Intersections = {year:IntPG for year in Years}


#Crash Dataset
CrashData  = {y:os.path.abspath('C:\Users\mr068144\Downloads\IL Crash Data\\' + str(y) + '\\AggregateFiles' +str(y)+ '.mdb'
                          '\\CrashExtract_' + str(y) + '_GIS') for y in Years}


Fields = ['AADT',
          'O_SHD1_TYP','O_SHD1_WTH','O_SHD2_TYP','O_SHD2_WTH',
          'I_SHD1_TYP','I_SHD1_WTH','I_SHD2_TYP','I_SHD2_WTH',
          'LNS','LN_WTH','MED_TYP','MED_WTH','SURF_WTH','URBAN','PG','Radius','CurveLen']
#Fields = ['AADT','URBAN','PG']
RouteID = 'INVENTORY'
BMP = 'BEG_STA'
EMP = 'END_STA'

# Geocoded projects
HSIP_Int  = Project_GDB + '\\HSIP_Int'
HSIP_Seg  = Project_GDB + '\\HSIP_Seg'

# HSIP Dataset
HSIP_Data = Project_DIR + '\\HSIP_AllApprovedProjects.xlsm'

# SPFs
SPF_Data = Project_DIR + '\\SPF Coefficients.xlsx'

2018-06-29 13:42:49


In [4]:
# Reading SPF Data
SPF_DF = pd.read_excel(SPF_Data,sheetname='Summary')
SPF_DF['PGNumber'] = [int(s.split('PeerGroup ')[1][:2]) for s in SPF_DF.PG]

In [3]:
# Reading the HSIP Applications Data
print(strftime("%Y-%m-%d %H:%M:%S"))
HSIP_DF = hsmpy.il.ReadHSIPData(HSIP_Data,Years)
HSIP_DF.index = HSIP_DF['HSIP ID']
for per in ['Before','After']:
    for sev in ['K','A','B']:
        for met in ['OC','EC']:
            HSIP_DF['_'.join([per[0],sev,met])] = 0.0
HSIP_DF['EUAC'] = 0.0
HSIP_DF['EUAB_OC'] = 0.0
HSIP_DF['EUAB_EC'] = 0.0
HSIP_DF['BC_OC'] = 0.0
HSIP_DF['BC_EC'] = 0.0

2018-06-25 10:35:43


In [80]:
# Reading Geocoded HSIP Data
print(strftime("%Y-%m-%d %H:%M:%S"))
print('Reading Intersections')
s1 = hsmpy.common.PrintSummary(HSIP_Int)
PN1 = {int(r.getValue('HSIP_ID')):0 for r in arcpy.SearchCursor(HSIP_Int)}
for r in arcpy.SearchCursor(HSIP_Int):
    hsipid = int(r.getValue('HSIP_ID'))
    PN1[hsipid] += 1
NumInt_Geo = []
for i,hsip in HSIP_DF.iterrows():
    if hsip['HSIP ID'] in PN1.keys():
        NumInt_Geo.append(PN1[hsip['HSIP ID']])
    else:
        NumInt_Geo.append(0)
HSIP_DF['NumInt_Geo'] = NumInt_Geo

print('Reading Segments')
s2 = hsmpy.common.PrintSummary(HSIP_Seg)
print('Total Mileage: {:0.2f}'.format(sum([r.getValue('Shape').length/5280 for r in arcpy.SearchCursor(HSIP_Seg)])))
PN2 = {int(r.getValue('HSIP_ID')):{'Segs':0,'Mileage':0} for r in arcpy.SearchCursor(HSIP_Seg)}
for r in arcpy.SearchCursor(HSIP_Seg):
    hsipid = int(r.getValue('HSIP_ID'))
    PN2[hsipid]['Segs'] += 1
    PN2[hsipid]['Mileage'] += r.getValue('Shape').length/5280
NumSeg_Geo = []
MilageSeg_Geo = []
for i,hsip in HSIP_DF.iterrows():
    if hsip['HSIP ID'] in PN2.keys():
        NumSeg_Geo.append(PN2[hsip['HSIP ID']]['Segs'])
        MilageSeg_Geo.append(PN2[hsip['HSIP ID']]['Mileage'])
    else:
        NumSeg_Geo.append(0)
        MilageSeg_Geo.append(0)
HSIP_DF['NumSeg_Geo'] = NumSeg_Geo
HSIP_DF['MilageSeg_Geo'] = MilageSeg_Geo

pn = PN1.keys()
pn.extend(PN2.keys())
PN  = {p:0 for p in pn}.keys()
PN.sort()
print('Total Geocoded HSIP Applications {}'.format(len(PN)))  
df = HSIP_DF[['HSIP ID','ApprovedAmt','Systematic Improvements','Is Segment','Is Intersection','NumInt_Geo','NumSeg_Geo','MilageSeg_Geo']]
df = df.sort_values(by=['MilageSeg_Geo','NumInt_Geo'],ascending=False)
df.to_html("deleteme.html")
url = "http://localhost:8888/tree/Downloads/HSIP_TrackingTool/deleteme.html"
#webbrowser.open(url,new=2)

2018-06-08 17:24:12
Reading Intersections
Type: Point
Columns: 4 x Rows: 252
[u'HSIP_ID', u'OBJECTID', u'Shape', u'SiteID']
Reading Segments
Type: Polyline
Columns: 22 x Rows: 396
[u'BCRatio', u'CompDate', u'CountMsr', u'District', u'EstCost', u'HSIPAward', u'HSIP_ID', u'Imps', u'IncInt', u'IncSeg', u'Lat', u'LettDate', u'Local', u'Lon', u'OBJECTID', u'PCrashTyp', u'Shape', u'Shape_Length', u'SiteID', u'Stratg', u'Systemic', u'TotAward']
Total Mileage: 1978.72
Total Geocoded HSIP Applications 516


In [ ]:
# Adding SiteID to HSIP Locations
print(strftime("%Y-%m-%d %H:%M:%S"))
def AddSiteID(FC,Start):
    try: arcpy.AddField_management(FC,'SiteID','SHORT')
    except: pass
    uc = arcpy.UpdateCursor(FC)
    i = Start
    for r in uc:
        r.setValue('SiteID',i)
        i += 1
        uc.updateRow(r)
    del uc, r
AddSiteID(HSIP_Seg,0)
AddSiteID(HSIP_Int,1000)

In [ ]:
#Convert MDBs to GDBs
print(strftime("%Y-%m-%d %H:%M:%S"))
MDBPath = AnalysisDir
GDBPath = AnalysisDir2
MDBList = [os.path.join(MDBPath, f) for f in os.listdir(MDBPath) if os.path.isfile(os.path.join(MDBPath, f)) and f.split('.')[1]=='mdb']
SP = []
for mdb in MDBList:
    gdb = GDBPath  + '\\' + os.path.basename(mdb).split('.')[0] + '.gdb'
    SP.append({'Process':hsmpy.common.CON_ConvertMDBtoGDB(WDir=GDBPath,HSMPY_PATH=HSMPY_PATH,gdb = gdb,mdb=mdb),
              'Title':os.path.basename(mdb).split('.')[0],'Printed':False,'LastStatus':'Initiated'})
    SP = hsmpy.common.WaitIfNecessary(SP,10,2)
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
#Extract Intersections from Roadways
print(strftime("%Y-%m-%d %H:%M:%S"))
hsmpy.network.PrintSegSummary(HSIP_Seg,'Initial Segments')
HSIP_Seg_exc = hsmpy.common.CreateOutPath(MainFile = HSIP_Seg,appendix='Exc',Extension='')
hsmpy.network.ExtractIntFromSeg(HSIP_Seg,Intersections[2016],250,HSIP_Seg_exc)
hsmpy.network.PrintSegSummary(HSIP_Seg_exc,'Excluded Segments')

In [8]:
# How many features are in each MDB
print(strftime("%Y-%m-%d %H:%M:%S"))
MDBLen = pd.DataFrame(columns = ['HSIP ID','Number of FeatureClasses','Datasets'])
i = 0
for p in PN:
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    d = [datasets for root, dirs, datasets in arcpy.da.Walk(MDB)][0]
    MDBLen.loc[i] = [p,len(d),';'.join([str(j) for j in d])]
    i += 1
#display(MDBLen)
# Monitor Incomplete MDBs
s = pd.DataFrame(MDBLen['Number of FeatureClasses'].value_counts())
display(s)
display(MDBLen.loc[MDBLen['Number of FeatureClasses']!=s.index[0]])

2018-06-05 08:55:04


,Number of FeatureClasses
62.0,229
50.0,199
74.0,40
2.0,17
69.0,8
72.0,3
57.0,3
70.0,2
61.0,2
46.0,2


,HSIP ID,Number of FeatureClasses,Datasets
0,201112004,74.0,Seg_201112004;Int_201112004;HWY2005_route;HWY2...
2,201201006,61.0,Seg_201201006;Int_201201006;HWY2005_route;HWY2...
4,201202002,74.0,Seg_201202002;Int_201202002;HWY2005_route;HWY2...
10,201203007,74.0,Seg_201203007;Int_201203007;HWY2005_route;HWY2...
11,201203008,74.0,Seg_201203008;Int_201203008;HWY2005_route;HWY2...
13,201203010,74.0,Seg_201203010;Int_201203010;HWY2005_route;HWY2...
14,201203011,61.0,Seg_201203011;Int_201203011;HWY2010_route;HWY2...
16,201203013,74.0,Seg_201203013;Int_201203013;HWY2005_route;HWY2...
17,201203015,74.0,Seg_201203015;Int_201203015;HWY2005_route;HWY2...
18,201203016,74.0,Seg_201203016;Int_201203016;HWY2005_route;HWY2...


In [ ]:
# Compact MDBs
print(strftime("%Y-%m-%d %H:%M:%S"))
BeforeSize = []
AfterSize = []
for p in PN:
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    if os.path.exists(MDB):
        BeforeSize.append(os.path.getsize(MDB))
        arcpy.Compact_management(MDB)
        AfterSize.append(os.path.getsize(MDB))
print('{} MDB Files reduced from {:,.2f} MB to {:,.2f} MB'.format(len(BeforeSize),sum(BeforeSize)/1048576.0,sum(AfterSize)/1048576.0))
print('Average MDB Size after: {:,.2f} MB'.format(sum(AfterSize)/1048576.0/max(1,len(AfterSize))))
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Create MDB Files for each project
print(strftime("%Y-%m-%d %H:%M:%S"))
#PN = ['201112004','201201002']
HSIP_Seg_exc = HSIP_Seg + '_Exc'
SP_CreateMDB = []
for p in PN:
    p = str(p)
    SP_CreateMDB.append({'Process':hsmpy.il.CON_CreateMDBforHSIP(AnalysisDir,HSMPY_PATH,p,HSIP_Seg_exc,HSIP_Int),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_CreateMDB = WaitIfNecessary(SP_CreateMDB,MaxOpenProcesses,Frequency = 2)

In [ ]:
# Adding Info needed from HSIP to Geocoded Sites:
# Predominent Crash Type, All Selected Improvements, 
print(strftime("%Y-%m-%d %H:%M:%S"))
HSIPF = ['Letting Date','Completion Date',
         'Pred. Crash Type','All Selected Improvements','Strategies','Countermeasures'
         'HSIP AWARD AMOUNT','Estimated Project Cost','Total Award Amount','HSIP AWARD AMOUNT','Benefit Cost Ratio'
         ]
GISFields = [
    {'HSIPField':'District'                  ,'GISField':'District' ,'Type':'Short' },
    {'HSIPField':'Is Segment'                ,'GISField':'IncSeg'   ,'Type':'TEXT' },
    {'HSIPField':'Is Intersection'           ,'GISField':'IncInt'   ,'Type':'TEXT' },
    {'HSIPField':'Is Local Project'          ,'GISField':'Local'    ,'Type':'TEXT' },
    {'HSIPField':'Systematic Improvements'   ,'GISField':'Systemic' ,'Type':'TEXT' },
    {'HSIPField':'Strategies'                ,'GISField':'Stratg'   ,'Type':'TEXT'  },
    {'HSIPField':'Countermeasures'           ,'GISField':'CountMsr' ,'Type':'TEXT'  },
    {'HSIPField':'All Selected Improvements' ,'GISField':'Imps'     ,'Type':'TEXT'  },
    {'HSIPField':'Pred. Crash Type'          ,'GISField':'PCrashTyp','Type':'TEXT'  },
    {'HSIPField':'Letting Date'              ,'GISField':'LettDate' ,'Type':'Date'  },
    {'HSIPField':'Completion Date'           ,'GISField':'CompDate' ,'Type':'Date'  },
    {'HSIPField':'Estimated Project Cost'    ,'GISField':'EstCost'  ,'Type':'Long'  },
    {'HSIPField':'Total Award Amount'        ,'GISField':'TotAward' ,'Type':'Long'  },
    {'HSIPField':'HSIP AWARD AMOUNT'         ,'GISField':'HSIPAward','Type':'Long'  },
    {'HSIPField':'Benefit Cost Ratio'        ,'GISField':'BCRatio'  ,'Type':'Double'},
    {'HSIPField':'Latitude'                  ,'GISField':'Lat'  ,'Type':'Double'},
    {'HSIPField':'Longitude'                 ,'GISField':'Lon'  ,'Type':'Double'},
]
SegDropList = [field['GISField'] for field in GISFields if field['GISField'] in [f.name for f in arcpy.ListFields(HSIP_Seg)]]
IntDropList = [field['GISField'] for field in GISFields if field['GISField'] in [f.name for f in arcpy.ListFields(HSIP_Int)]]
if not SegDropList == []:
    arcpy.DeleteField_management(HSIP_Seg,SegDropList)
if not IntDropList == []:
    arcpy.DeleteField_management(HSIP_Int,IntDropList)

for field in GISFields:
    arcpy.AddField_management(HSIP_Seg,field['GISField'],field['Type'],field_length=500)
    arcpy.AddField_management(HSIP_Int,field['GISField'],field['Type'],field_length=500) 

UC = arcpy.UpdateCursor(HSIP_Seg)
for r in UC:
    HSIPID = int(r.getValue('HSIP_ID'))
    for field in GISFields:
        try:
            if field['Type'] == 'Date':
                k = HSIP_DF[field['HSIPField']][HSIP_DF['HSIP ID']==HSIPID]
                k = str(list(k)[0])
                k = datetime.strptime(k,'%Y-%m-%d %H:%M:%S')
            else:
                k = HSIP_DF[field['HSIPField']][HSIP_DF['HSIP ID']==HSIPID]
                k = list(k)[0]
            r.setValue(field['GISField'],k)
            UC.updateRow(r)
        except:
            pass
del UC
display(HTML(hsmpy.common.AttributeTabletoDF(HSIP_Seg).to_html()))
UC = arcpy.UpdateCursor(HSIP_Int)
for r in UC:
    HSIPID = int(r.getValue('HSIP_ID'))
    for field in GISFields:
        try:
            if field['Type'] == 'Date':
                k = HSIP_DF[field['HSIPField']][HSIP_DF['HSIP ID']==HSIPID]
                k = str(list(k)[0])
                k = datetime.strptime(k,'%Y-%m-%d %H:%M:%S')
            else:
                k = HSIP_DF[field['HSIPField']][HSIP_DF['HSIP ID']==HSIPID]
                k = list(k)[0]
            r.setValue(field['GISField'],k)
            UC.updateRow(r)
        except:
            pass
del UC
display(HTML(hsmpy.common.AttributeTabletoDF(HSIP_Int).to_html()))

In [12]:
# Add Base Data to each MDB
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
SelectionExtent = "0.5 Miles"
SP_AddBaseData =[]
for p in PN:
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    SegFC = MDB + '\\Seg_' + p
    IntFC = MDB + '\\Int_' + p
    SP_AddBaseData.append({'Process':hsmpy.il.CON_AddBaseData(AnalysisDir,HSMPY_PATH,[SegFC,IntFC],
                                          Years,IRIS_Route,IRIS_Tab,Intersections,
                                          MDB,SelectionExtent,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_AddBaseData = WaitIfNecessary(SP_AddBaseData,MaxOpenProcesses,Frequency = 2)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-06-01 12:18:20
HSIP ID: 201112004: 0
HSIP ID: 201201002: 0
HSIP ID: 201201006: 0
HSIP ID: 201201007: 0
HSIP ID: 201203002: 0
HSIP ID: 201203003: 0
HSIP ID: 201202004: 0
HSIP ID: 201202002: 0
HSIP ID: 201203004: 0
HSIP ID: 201202003: 0
HSIP ID: 201203011: 1
HSIP ID: 201203015: 0
HSIP ID: 201203009: 0
HSIP ID: 201203013: 0
HSIP ID: 201203018: 0
HSIP ID: 201203007: 0
HSIP ID: 201203012: 0
HSIP ID: 201203016: 0
HSIP ID: 201203008: 0
HSIP ID: 201203010: 0
HSIP ID: 201204003: 0
HSIP ID: 201205001: 0
HSIP ID: 201205002: 0
HSIP ID: 201205004: 0
HSIP ID: 201207001: 0
HSIP ID: 201208001: 0
HSIP ID: 201205003: 0
HSIP ID: 201209001: 0
HSIP ID: 201209002: 0
HSIP ID: 201207002: 0
HSIP ID: 201209003: 0
HSIP ID: 201209004: 0
HSIP ID: 201209005: 0
HSIP ID: 201209006: 0
HSIP ID: 201211005: 0
HSIP ID: 201211009: 0
HSIP ID: 201212001: 0
HSIP ID: 201212002: 0
HSIP ID: 201212003: 0
HSIP ID: 201209008: 0
HSIP ID: 201212007: 0
HSIP ID: 201212010: 0
HSIP ID: 201212011: 0
HSIP ID: 201212012: 0
HSIP ID: 201

HSIP ID: 201410382: 0
HSIP ID: 201410384: 0
HSIP ID: 201410376: 0
HSIP ID: 201410388: 0
HSIP ID: 201410386: 0
HSIP ID: 201410390: 0
HSIP ID: 201410391: 0
HSIP ID: 201410393: 0
HSIP ID: 201410394: 0
HSIP ID: 201410392: 0
HSIP ID: 201410395: 0
HSIP ID: 201410396: 0
HSIP ID: 201410399: 0
HSIP ID: 201410400: 0
HSIP ID: 201410401: 0
HSIP ID: 201411005: 0
HSIP ID: 201411003: 0
HSIP ID: 201412002: 0
HSIP ID: 201411007: 0
HSIP ID: 201411008: 0
HSIP ID: 201412003: 0
HSIP ID: 201412004: 0
HSIP ID: 201412005: 0
HSIP ID: 201412008: 0
HSIP ID: 201412009: 0
HSIP ID: 201502001: 1
HSIP ID: 201412012: 0
HSIP ID: 201412011: 0
HSIP ID: 201502004: 1
HSIP ID: 201501003: 0
HSIP ID: 201501005: 0
HSIP ID: 201501004: 0
HSIP ID: 201501007: 0
HSIP ID: 201502003: 0
HSIP ID: 201502002: 0
HSIP ID: 201502005: 0
HSIP ID: 201502009: 1
HSIP ID: 201502010: 0
HSIP ID: 201502014: 0
HSIP ID: 201503002: 0
HSIP ID: 201503009: 0
HSIP ID: 201503003: 0
HSIP ID: 201503010: 0
HSIP ID: 201503011: 0
HSIP ID: 201503013: 0
HSIP ID: 2

In [9]:
# Import Roadway and Int Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
XY_Tolerance = "70 Feet"
SP_Att =[]
for p in PN:
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    SegFC = MDB + '\\Seg_' + p
    IntFC = MDB + '\\Int_' + p
    RteFN = {year:os.path.join(MDB,'HWY' + str(year) + '_route') for year in Years}
    IntFN = {year:os.path.join(MDB,'HWY' + str(year) + '_inter') for year in Years}
    TabFN = {year:os.path.join(MDB,'HWY' + str(year) + '_table') for year in Years}
    NumSeg = 0
    NumInt = 0
    try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
    except: pass
    try: NumInt = int(str(arcpy.GetCount_management(IntFC)))
    except: pass
    #print('HSIP ID: {}, Number of Segs: {}, Number of Int: {}'.format(p,NumSeg,NumInt))
    if NumSeg > 0:
        RoadwayData = {year:hsmpy.common.CreateOutPath(MainFile=SegFC,appendix=str(year),Extension='') for year in Years}
        SP_Att.append({'Process':hsmpy.network.CON_ImportRoadwayData_Temporal(AnalysisDir,HSMPY_PATH,
                                                                              SegFC,RteFN,TabFN,
                                                                              Fields,RoadwayData,
                                                                              RouteID,BMP,EMP,XY_Tolerance,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    if NumInt > 0:
        IntPoints = {year:hsmpy.common.CreateOutPath(MainFile=IntFC,appendix=str(year) + '_points',Extension='') for year in Years}
        IntTables = {year:hsmpy.common.CreateOutPath(MainFile=IntFC,appendix=str(year) + '_tables',Extension='') for year in Years}
        SP_Att.append({'Process':hsmpy.network.CON_ImportIntData_Temporal(AnalysisDir,HSMPY_PATH,
                                                                   IntFC,IntFN,
                                                                   RteFN,RouteID,BMP,EMP,TabFN,Fields,
                                                                   IntPoints,IntTables,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_Att = WaitIfNecessary(SP_Att,MaxOpenProcesses,Frequency = 2)    
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-06-03 15:31:48
HSIP ID: 201201002: 0
HSIP ID: 201201006: 0
HSIP ID: 201201007: 0
HSIP ID: 201202003: 0
HSIP ID: 201112004: 0
HSIP ID: 201202004: 0
HSIP ID: 201202002: 0
HSIP ID: 201203002: 0
HSIP ID: 201203003: 0
HSIP ID: 201203004: 0
HSIP ID: 201203009: 0
HSIP ID: 201203007: 0
HSIP ID: 201203011: 0
HSIP ID: 201203008: 0
HSIP ID: 201203012: 0
HSIP ID: 201203010: 0
HSIP ID: 201203013: 0
HSIP ID: 201203018: 0
HSIP ID: 201203015: 0
HSIP ID: 201203016: 0
HSIP ID: 201204003: 0
HSIP ID: 201205001: 0
HSIP ID: 201207001: 0
HSIP ID: 201205002: 0
HSIP ID: 201205003: 0
HSIP ID: 201207002: 0
HSIP ID: 201208001: 0
HSIP ID: 201205004: 0
HSIP ID: 201209001: 0
HSIP ID: 201209002: 0
HSIP ID: 201209003: 0
HSIP ID: 201209004: 0
HSIP ID: 201209008: 0
HSIP ID: 201209005: 0
HSIP ID: 201212001: 0
HSIP ID: 201209006: 0
HSIP ID: 201211005: 0
HSIP ID: 201212007: 0
HSIP ID: 201211009: 0
HSIP ID: 201212002: 0
HSIP ID: 201212011: 0
HSIP ID: 201212003: 0
HSIP ID: 201212010: 0
HSIP ID: 201212015: 0
HSIP ID: 201

HSIP ID: 201410384: 0
HSIP ID: 201410395: 0
HSIP ID: 201410388: 0
HSIP ID: 201410396: 0
HSIP ID: 201410390: 0
HSIP ID: 201410399: 0
HSIP ID: 201411003: 0
HSIP ID: 201411008: 0
HSIP ID: 201410400: 0
HSIP ID: 201410401: 0
HSIP ID: 201411005: 0
HSIP ID: 201411007: 0
HSIP ID: 201412002: 0
HSIP ID: 201412003: 0
HSIP ID: 201412004: 0
HSIP ID: 201412005: 0
HSIP ID: 201412008: 0
HSIP ID: 201412011: 0
HSIP ID: 201412012: 0
HSIP ID: 201501004: 0
HSIP ID: 201412009: 0
HSIP ID: 201501007: 0
HSIP ID: 201502002: 0
HSIP ID: 201501003: 0
HSIP ID: 201502005: 0
HSIP ID: 201501005: 0
HSIP ID: 201502010: 0
HSIP ID: 201502003: 0
HSIP ID: 201502014: 0
HSIP ID: 201502009: 1
HSIP ID: 201503002: 0
HSIP ID: 201503003: 0
HSIP ID: 201503010: 0
HSIP ID: 201503011: 0
HSIP ID: 201503013: 0
HSIP ID: 201503014: 0
HSIP ID: 201503009: 0
HSIP ID: 201504001: 0
HSIP ID: 201504010: 0
HSIP ID: 201504011: 0
HSIP ID: 201504012: 0
HSIP ID: 201505021: 1
HSIP ID: 201505009: 0
HSIP ID: 201505014: 0
HSIP ID: 201505015: 0
HSIP ID: 2

In [ ]:
# Add Observed Crashes
#[p['Process'].wait() for p in SP_Att]
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
Seg_Buffer_Distance = '50 Feet'
Int_Buffer_Distance = '250 Feet'
SP_ObsCrash =[]
for p in PN[150:]:
    Sites = []
    for year in Years:
        p = str(p)
        FN = 'HSIP_' + p + '_GIS.mdb'
        MDB = os.path.join(AnalysisDir,FN)
        SegFC = MDB + '\\Seg_' + p + '_' + str(year)
        IntFC = MDB + '\\Int_' + p + '_' + str(year) + '_points'
        NumSeg = 0
        NumInt = 0
        try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
        except: pass
        try: NumInt = int(str(arcpy.GetCount_management(IntFC)))
        except: pass
        #if NumSeg > 0 or NumInt > 0:
            #print('HSIP ID: {}, Year: {}, Number of Segs: {}, Number of Int: {}'.format(p,year,NumSeg,NumInt))
        if NumSeg > 0:
            SegCrashes = hsmpy.common.CreateOutPath(MainFile= SegFC,appendix=str('Crash'),Extension='')
            Sites.append({'year':year,'Input':SegFC,'Output':SegCrashes,'Buffer':Seg_Buffer_Distance})
        if NumInt > 0:
            IntCrashes = hsmpy.common.CreateOutPath(MainFile= IntFC,appendix=str('Crash'),Extension='')
            Sites.append({'year':year,'Input':IntFC,'Output':IntCrashes,'Buffer':Int_Buffer_Distance})
    SP_ObsCrash.append({'Process':hsmpy.crash.CON_AddObservedCrashes_Temporal(AnalysisDir,HSMPY_PATH,Sites,CrashData,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_ObsCrash = WaitIfNecessary(SP_ObsCrash,MaxOpenProcesses,Frequency = 2)    

In [ ]:
# Add Predicted Crashes
#[p['Process'].wait() for p in SP_ObsCrash]
SP_PrdCrash = []
for p in PN:
    #print(p)
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    SP_PrdCrash.append({'Process':hsmpy.il.CON_Seg_PC_HSIP(AnalysisDir,HSMPY_PATH,MDB,Years,SPF_Data,p),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_PrdCrash = WaitIfNecessary(SP_PrdCrash,15,Frequency = 2)  

In [ ]:
# Mileage report
print(strftime("%Y-%m-%d %H:%M:%S"))
SiteIDDict = {r.getValue('SiteID'):{'Initial':r.getValue('Shape').length/5280,'Exc':0} for r in arcpy.SearchCursor(HSIP_Seg)}
HSIP_Seg_exc = HSIP_Seg + '_Exc'
RoadwayData = {year:os.path.join(Project_GDB,'Seg_'+str(year)) for year in Years}
for ID in SiteIDDict:
    SiteIDDict[ID].update({y:0 for y in Years})
for r in arcpy.SearchCursor(HSIP_Seg_exc):
    SiteIDDict[r.getValue('SiteID')]['Exc'] += r.getValue('Shape').length/5280 
df = pd.DataFrame(columns=['Initial','Exc'])
for ID in SiteIDDict:
    df.loc[ID] = [SiteIDDict[ID]['Initial'],SiteIDDict[ID]['Exc']]
IDList = [ID for ID in SiteIDDict]
IDList.sort()
for year in Years:
    try:
        for r in arcpy.SearchCursor(RoadwayData[year]):
            SiteIDDict[r.getValue('SiteID')][year] += r.getValue('Shape').length/5280 
            #print(r.getValue('SiteID'),r.getValue('Shape').length/5280 )
        df[year] = [SiteIDDict[ID][year] for ID in IDList]
    except:
        pass
display(df)

In [ ]:
# Create Summary Sheet
SP_ExcelSheet = []
for p in PN:
    if not(len(HSIP_DF[HSIP_DF['HSIP ID'] ==  int(p)]['BeforePeriod'].item())>0 and len(HSIP_DF[HSIP_DF['HSIP ID'] ==  int(p)]['AfterPeriod'].item())>0):
        print('{}: Before-After Periods are not defined'.format(p))
        continue
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    FN = os.path.join(AnalysisDir,'HSIP_'+str(p) + '.xlsx')
    try: os.remove(FN)
    except: pass
    SP_ExcelSheet.append({'Process':hsmpy.il.CON_ExcelSheetSummary(AnalysisDir,HSMPY_PATH,MDB,Years,FN,HSIP_Data,p,Fields),
                        'HSIP_ID':p,'Printed':False,'LastStatus':'Initiated'})
    SP_ExcelSheet = WaitIfNecessary(SP_ExcelSheet,15,Frequency = 2)  

In [136]:
Years = range(2005,2017)
def AdjustExtend(ext,distance):
    ext.XMax += distance
    ext.XMin -= distance
    ext.YMax += distance
    ext.YMin -= distance   
    return(ext)
def AddFCtoDataFrame(DF,FC,Position = 'AUTO_ARRANGE'):
    layer = ''
    try:
        LayerName = os.path.basename(FC)
        arcpy.Delete_management(LayerName)
        arcpy.MakeFeatureLayer_management(in_features=FC,out_layer=LayerName)
        layer = arcpy.mapping.Layer(LayerName)
        arcpy.mapping.AddLayer(DF,layer,Position)
    except:
        pass
    return(layer)
def AddFCtoDataFrameGroup(DF,Group,FC,Position = 'AUTO_ARRANGE'):
    layer = ''
    try:
        LayerName = os.path.basename(FC)
        arcpy.Delete_management(LayerName)
        arcpy.MakeFeatureLayer_management(in_features=FC,out_layer=LayerName)
        layer = arcpy.mapping.Layer(LayerName)
        arcpy.mapping.AddLayerToGroup(DF,Group,layer,Position)
    except:
        #print(FC)
        pass
    return(layer)

MXD = os.path.join(AnalysisDir,'Blank_Projected.mxd')
SegSymbology = os.path.join(AnalysisDir,'SegSymbology.lyr')
IntSymbology = os.path.join(AnalysisDir,'IntSymbology.lyr')
for p in PN:
    p = str(p)
    FN = 'HSIP_' + p + '_GIS.mdb'
    MDB = os.path.join(AnalysisDir,FN)
    SegFC = MDB + '\\Seg_' + p
    IntFC = MDB + '\\Int_' + p
    #arcpy.RecalculateFeatureClassExtent_management (SegFC)
    #arcpy.RecalculateFeatureClassExtent_management (IntFC)
    RteFN = {year:os.path.join(MDB,'HWY' + str(year) + '_route') for year in Years}
    IntFN = {year:os.path.join(MDB,'HWY' + str(year) + '_inter') for year in Years}
    TabFN = {year:os.path.join(MDB,'HWY' + str(year) + '_table') for year in Years}

    NumSeg = 0
    NumInt = 0
    try: NumSeg = int(str(arcpy.GetCount_management(SegFC)))
    except: pass
    try: NumInt = int(str(arcpy.GetCount_management(IntFC)))
    except: pass
    
    p = str(p)
    mxd = arcpy.mapping.MapDocument(MXD)
    df = arcpy.mapping.ListDataFrames(mxd)[0]
    LG_BaseInt = arcpy.mapping.ListLayers(mxd,"Base Intersection Data",df)[0]
    LG_BaseSeg = arcpy.mapping.ListLayers(mxd,"Base Roadway Data",df)[0]
    LG_Sites   = arcpy.mapping.ListLayers(mxd,"Geocoded Sites",df)[0]
    LG_SegAtt  = arcpy.mapping.ListLayers(mxd,"Segment Attribute Data",df)[0]
    LG_IntAtt  = arcpy.mapping.ListLayers(mxd,"Intersection Att Data",df)[0]
    LG_Crash   = arcpy.mapping.ListLayers(mxd,"Crash Data",df)[0]

    SegLayer = AddFCtoDataFrameGroup(df,LG_Sites,SegFC)
    IntLayer = AddFCtoDataFrameGroup(df,LG_Sites,IntFC)

    for year in Years:
        AddFCtoDataFrameGroup(df,LG_BaseSeg,RteFN[year])
        AddFCtoDataFrameGroup(df,LG_BaseInt,IntFN[year])

        AddFCtoDataFrameGroup(df,LG_SegAtt,SegFC + '_' + str(year))
        AddFCtoDataFrameGroup(df,LG_IntAtt,IntFC + '_' + str(year) + '_points' )
        AddFCtoDataFrameGroup(df,LG_Crash,SegFC + '_' + str(year) + '_Crash')
        AddFCtoDataFrameGroup(df,LG_Crash,IntFC + '_' + str(year) + '_points_Crash' )
        try:arcpy.Delete_management(MDB + '\\HWY' + str(year) + '_Tab')
        except: pass
        arcpy.CreateRelationshipClass_management(origin_table = RteFN[year], 
                                                     destination_table = TabFN[year], 
                                                     out_relationship_class = MDB + '\\HWY' + str(year) + '_Tab', 
                                                     relationship_type = 'SIMPLE', 
                                                     forward_label = 'Table', 
                                                     backward_label = 'Route', 
                                                     message_direction = 'BOTH', 
                                                     cardinality = 'ONE_TO_MANY', 
                                                     attributed = 'NONE', 
                                                     origin_primary_key = 'INVENTORY', 
                                                     origin_foreign_key = 'INVENTORY')
    
    if NumSeg>0:
        ext = SegLayer.getExtent()
    if NumInt>0:
        ext = IntLayer.getExtent()
    
    df.spatialReference = arcpy.SpatialReference(102672)
    ext = ext.projectAs(arcpy.SpatialReference(102672))
    ext = AdjustExtend(ext,1000)
    df.extent = ext
    arcpy.ApplySymbologyFromLayer_management (SegLayer, SegSymbology)
    arcpy.ApplySymbologyFromLayer_management (IntLayer, IntSymbology)

    arcpy.RefreshActiveView()
    
    ImageName = 'HSIP_' + p + '_Map.jpg'
    arcpy.mapping.ExportToJPEG(mxd, os.path.join(AnalysisDir, ImageName), df,
                           df_export_width=1000,
                           df_export_height=1000,
                           world_file=True)
    mxd.saveACopy (os.path.join(AnalysisDir,'HSIP_' + p + '.mxd'), '10.3')
    try:os.remove(os.path.join(AnalysisDir, 'HSIP_' + p + '_Map.jgw'))
    except:pass
    print(p)  

201112004
201201002
201201006
201201007
201202002
201202003
201202004
201203002
201203003
201203004
201203007
201203008
201203009
201203010
201203011
201203012
201203013


AttributeError: DataFrameObject: Error in executing ExportToJPEG

In [72]:
XLSPath = AnalysisDir
XLSList = [os.path.join(XLSPath, f) for f in os.listdir(XLSPath) if os.path.isfile(os.path.join(XLSPath, f)) and f.split('.')[1]=='xlsx' and 'HSIP_' in f.split('.')[0]]    
for xls in XLSList:
    p = int(os.path.basename(xls).split('_')[1].split('.')[0])
    df3 = pd.read_excel(xls,'RawCrash')
    PlotCrashType(df3,os.path.join(AnalysisDir,'HSIP_' + str(p) + '_CrashType.png'),p)
    print(p)

201112004
201201002
201201007
201203010
201309010
201410002
201410003
201410004
201410005
201410006
201410007
201410008
201410010
201410011
201410012
201410014
201410015
201410016
201410017
201410019
201410020
201410021
201410033
201410044
201410057
201410058
201410059
201410060
201410062
201410066
201410067
201410068
201410069
201410071
201410073
201410075
201410077
201410078
201410079
201410083
201410086
201410089
201410091
201410092
201410094
201410095
201410097
201410099
201410100
201410101
201410112
201410113
201410114
201410115
201410116
201410117
201410118
201410119
201410122
201410124
201410126
201410128
201410129
201410130
201410134
201410135
201410137
201410141
201410142
201410143
201410144
201410145
201410146
201410147
201410150
201410151
201410152
201410153
201410166
201410167
201410168
201410169
201410170


IOError: [Errno 13] Permission denied: '\\\\CHCFPP01\\Proj\\ILDOT\\650511SAFETYPROGRAM\\4_WorkData\\WO12\\HSIP_TrackingTool\\HSIPTrackingGIS\\AnalysisFiles\\~$HSIP_201201007.xlsx'

In [82]:
def PlotCrashType(df3,FN,ID):
    left, width = .1, .85
    bottom, height = .1, .85
    right = left + width
    top = bottom + height    

    BYs = list(set(df3[df3.Period=='BeforePeriod']['Year']))
    BYs.sort()
    AYs = list(set(df3[df3.Period=='AfterPeriod']['Year']))
    AYs.sort()
    if len(BYs)==0 or len(AYs)==0:
        return
    df4 = df3[df3.Period == 'BeforePeriod']
    df5 = df3[df3.Period == 'AfterPeriod']
    df4 = hsmpy.il.CrashTypeDF(df4)
    for c in df4.columns:
        df4[c] = [float(v)/len(BYs) for v in df4[c]]
    df5 = hsmpy.il.CrashTypeDF(df5)
    for c in df5.columns:
        df5[c] = [float(v)/len(AYs) for v in df5[c]]
    DFBefore = df4
    DFAfter = df5
    ymin = 0
    ymax = max([max([max(DFBefore[c]) for c in DFBefore.columns]),
                max([max(DFAfter[c])  for c in DFAfter.columns ])])
    if ymax == 0:
        return
    ymax = ymax * 1.1
    Conv_Dict = {
        'animal':'ANM',
        'fixed object':'FO',
        'head on':'HO',
        'overturned':'OT',
        'other object':'OO',
        'other non collision':'ONC',
        'rear end':'RE',
        'angle':'ANG',
        'turning':'TUR',
        'sideswipe opposite direction':'SSWO',
        'sideswipe same direction':'SSWS',
        'parked motor vehicle':'PMV',
        'pedestrian':'PED'
    }
    plt.figure(figsize=(8,6))
    #plt.gcf().suptitle("HSIP ID: " + str(ID), fontsize=14)

    plt.subplot(211)
    
    #fig, ax = plt.subplots(figsize=(6,3))  
    #axes = plt.gca()
    plt.gca().set_ylim([ymin,ymax])    
    DFBefore = hsmpy.il.RemoveTotals(DFBefore)
    DFAfter = hsmpy.il.RemoveTotals(DFAfter)
    df = DFBefore
    df.columns = [Conv_Dict[c] for c in df.columns]
    sev = df.index
    margin_bottom = np.zeros(len(df.columns))
    
    #cmap = mcolors.LinearSegmentedColormap('redgreen',  [(0,'#FF0000'),(1,'#74C476')], 100)
    #colors = [cmap(i) for i in np.linspace(0, 1, 5)]
    colors = ["red", "brown","coral","yellow","green"]

    for num, s in enumerate(sev):
        values = list(df.loc[s])
        df.loc[s].plot.bar(ax=plt.gca(), stacked=True, 
                                    bottom = margin_bottom, label=s,color = colors[num])
        margin_bottom += values
    plt.legend(loc='upper left',frameon = False,fancybox=True)
    plt.grid()
    #plt.gca().text(.5,.9,'Before Period',horizontalalignment='center',transform=plt.gca().transAxes)
    plt.title("HSIP ID: " + str(ID) + '\n Before Period')
    plt.gca().text(right, top, str(len(BYs)) + ' Years: ' + str(BYs[0]) + '-' + str(BYs[-1]),
        horizontalalignment='right',
        verticalalignment='top',
        transform=plt.gca().transAxes)
    
    #plt.show()
    plt.subplot(212)
    #fig, ax = plt.subplots(figsize=(6,3))  
    plt.gca().set_ylim([ymin,ymax])    
    df = DFAfter
    df.columns = [Conv_Dict[c] for c in df.columns]
    sev = df.index
    margin_bottom = np.zeros(len(df.columns))

    for num, s in enumerate(sev):
        values = list(df.loc[s])
        df.loc[s].plot.bar(ax=plt.gca(), stacked=True, 
                                    bottom = margin_bottom, label=s,color = colors[num])
        margin_bottom += values
    #plt.legend(loc='center left')
    plt.grid()
    plt.title('After Period')
    plt.gca().text(right, top, str(len(AYs)) + ' Years: ' + str(AYs[0]) + '-' + str(AYs[-1]),
        horizontalalignment='right',
        verticalalignment='top',
        transform=plt.gca().transAxes)
    
    plt.tight_layout()
    plt.savefig(FN,transparent=False)
    plt.close()

In [7]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2005,2017)
MaxOpenProcesses = 15
Project_DIR = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool'
Project_GDB = Project_DIR + '\\2.Project_GDB.gdb'
AnalysisDir = os.path.join(Project_DIR,'4.Contracts_AnalysisFiles')
DataDir     = Project_DIR + '\\7.ReceivedData'
PythonDir   = Project_DIR + '\\6.Python_Codes'



# Geocoded projects
HSIP_Int  = Project_GDB + '\\WPPS_Int'
HSIP_Seg  = Project_GDB + '\\WPPS_Seg'

XML_Path     = r'C:\Users\mr068144\Downloads\HSIP_TrackingTool\From IDOT - HSIP Candidate Applications'
HSIP_Path    = DataDir + '\\HSIP_AllApprovedProjects.xlsm'
WPPS_Ob_Path = DataDir + '\\HSIP Contract and Obligation Data.xlsx'
WPPS_Pr_Path = DataDir + '\\HSIP Project Programmed Data.xlsx'
SPF_Data     = DataDir  + '\\SPF Coefficients.xlsx'

# Reading Input Data
print(strftime("%Y-%m-%d %H:%M:%S"))
XML_DF     = hsmpy3.il.HSIP_DF_From_XML(XML_Path)

print(strftime("%Y-%m-%d %H:%M:%S"))
HSIP_DF    = hsmpy3.il.CreateHSIPDataFrame(HSIP_Path,Years)
HSIP_DF = HSIP_DF[[(i in XML_DF.HSIPID) for i in HSIP_DF.index]]

print(strftime("%Y-%m-%d %H:%M:%S"))
WPPS_Ob_DF = hsmpy3.il.HSIP_DF_WPPS_Ob(WPPS_Ob_Path,HSIP_DF,XML_DF)
WPPS_Pr_DF = hsmpy3.il.HSIP_DF_WPPS_Pr(WPPS_Pr_Path,HSIP_DF,XML_DF)

print(strftime("%Y-%m-%d %H:%M:%S"))
HSIP_Joined = hsmpy3.il.Joined_HSIP_DF(XML_DF,HSIP_DF,WPPS_Ob_DF,WPPS_Pr_DF)

print(strftime("%Y-%m-%d %H:%M:%S"))
Cont_DF = hsmpy3.il.Joined_Contract_DF(HSIP_Joined,WPPS_Ob_DF,WPPS_Pr_DF,DataDir,Years)

print(strftime("%Y-%m-%d %H:%M:%S"))
Loc_DF = hsmpy3.il.Joined_Loc_DF(WPPS_Pr_DF)

print(strftime("%Y-%m-%d %H:%M:%S"))
SPF_DF = pd.read_excel(SPF_Data,sheetname='Summary')
SPF_DF['PGNumber'] = [int(s.split('PeerGroup ')[1][:2]) for s in SPF_DF.PG]

PN = list(set(Loc_DF['ContNum']))
PN.sort()

2018-06-29 14:45:29
2018-06-29 14:45:29


NameError: name 'hsmpy3' is not defined

In [3]:
#Aggregated Results
XLSPath = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\HSIP_Tracking_Tool\4.Contracts_AnalysisFiles'
XLSList = [os.path.join(XLSPath, f) for f in os.listdir(XLSPath) if os.path.isfile(os.path.join(XLSPath, f)) and f.split('.')[1]=='xlsx' and 'HSIP_' in f.split('.')[0]]
for xls in XLSList:
    p = str(os.path.basename(xls).split('_')[1].split('.')[0])
    df10 = pd.read_excel(xls,'BCSummary')
    df10.index = df10.Period
    #display(df10)
    for per in ['Before','After']:
        for sev in ['K','A','B']:
            for met in ['OC','EC']:
                try:
                    Cont_DF.set_value(p,'_'.join([per[0],sev,met]),df10.loc[per]['_'.join([sev,met])].item())
                except:pass
    for c in ['EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC']:
        try:
            Cont_DF.set_value(p,c,df10.loc['After'][c].item())
        except:pass

In [5]:
c = []
for per in ['Before','After']:
    for sev in ['K','A','B']:
        for met in ['OC','EC']:
            c.append('_'.join([per[0],sev,met]))
c.extend(['EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC'])
display(Cont_DF[Cont_DF.EUAC!=0][c])

NameError: name 'Cont_DF' is not defined

In [7]:
Cont_DF_Eval = Cont_DF[[len(Cont_DF.loc[i]['BeforePeriod'].split(';'))>=3 and
                        len(Cont_DF.loc[i]['AfterPeriod' ].split(';'))>=3 and
                        Cont_DF.loc[i]['CompletionAmount']>0
                        for i in Cont_DF.index]]
print('Total Projects with 3y B&A & TAA>0: {}'.format(len(HSIP_DF_Eval)))
HSIP_DF_Eval_Prc = HSIP_DF_Eval[(HSIP_DF_Eval['EUAB_OC']<>0)]
print('Total Projects with 3y B&A & TAA>0 and BC<>0: {}'.format(len(HSIP_DF_Eval_Prc)))
print('Total BC_OC: {}'.format(sum(HSIP_DF_Eval_Prc.EUAB_OC)/sum(HSIP_DF_Eval_Prc.EUAC)))
print('Total BC_OC: {}'.format(sum(HSIP_DF_Eval_Prc.EUAB_EC)/sum(HSIP_DF_Eval_Prc.EUAC)))

Total Projects with 3y B&A & TAA>0: 258
Total Projects with 3y B&A & TAA>0 and BC<>0: 52
Total BC_OC: 22.5494402678
Total BC_OC: nan


In [63]:
def HSIP_Aggr_District(HSIP_DF):
    Aggr_DF = pd.DataFrame()
    Index = ['District1','District2','District3','District4','District5','District6','District7','District8','District9']

    Aggr_DF['Tot']    = [len(HSIP_DF[[int(i[-1])==r['District'] for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['W Tot Award']    = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['Tot.Awrd.Amount']= [sum(HSIP_DF[[int(i[-1])==r['District'] and
                         r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]['Total Award Amount']) for i in Index]

    Aggr_DF['3Ys BA'] = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>3 and
                         len(r['AfterPeriod' ].split(';'))>3 and
                         r['Total Award Amount']>0
                                      for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['Proc_OC'] = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['Proc_OC_BC>1'] = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['BC_OC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['EUAC_OC'] = [sum(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in Index]

    Aggr_DF['EUAB_OC'] = [sum(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_OC']) for i in Index]

    Aggr_DF['BC_OC'] = [r['EUAB_OC']/(r['EUAC_OC'] or not r['EUAC_OC']) for i,r in Aggr_DF.iterrows()]

    Aggr_DF['Proc_EC'] = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['Proc_EC_BC>1'] = [len(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0  and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']) and r['BC_EC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in Index]

    Aggr_DF['EUAC_EC'] = [sum(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in Index]
    
    Aggr_DF['EUAB_EC'] = [sum(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_EC']) for i in Index]

    Aggr_DF['BC_EC'] = [r['EUAB_EC']/(r['EUAC_EC'] or not r['EUAC_EC']) for i,r in Aggr_DF.iterrows()]


    Aggr_DF['HSIP_ID_OC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in Index]

    Aggr_DF['HSIP_ID_EC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[int(i[-1])==r['District'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in Index]
    
    
    Aggr_DF.index = Index
    #Aggr_DF['Total'] = [sum([r[c] for c in ['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA','Proc','EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC']]) for i,r in Aggr_DF.iterrows()]
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    return(Aggr_DF)
Aggr_DF = HSIP_Aggr_District(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
District1,242.00,151.00,153594599.98,53.00,19.00,12.00,1686052.25,72826258.94,43.19,17.00,12.00,1475357.78,43730418.23,29.64,201203010;201410002;201410003;201410004;201410...,201203010;201410002;201410003;201410004;201410...
District2,80.00,61.00,45481308.46,26.00,11.00,8.00,575725.71,11371613.11,19.75,11.00,8.00,575725.71,2559199.87,4.45,201112004;201410057;201410069;201410075;201410...,201112004;201410057;201410069;201410075;201410...
District3,61.00,48.00,53687022.64,6.00,3.00,3.00,181313.39,6082425.31,33.55,2.00,1.00,127932.13,255451.20,2.00,201410097;201410100;201410101,201410097;201410100
District4,65.00,42.00,34588725.84,14.00,9.00,5.00,1312233.28,5210829.33,3.97,8.00,2.00,1260026.45,346683.97,0.28,201201002;201410112;201410113;201410115;201410...,201201002;201410112;201410113;201410116;201410...
District5,51.00,32.00,44258712.41,8.00,4.00,2.00,224500.22,2299333.27,10.24,3.00,1.00,184373.43,-1052187.98,-5.71,201309010;201410134;201410135;201410137,201309010;201410134;201410135
District6,115.00,75.00,35373997.80,20.00,5.00,2.00,410657.15,2683189.02,6.53,0.00,0.00,0.00,0.00,0.00,201410166;201410167;201410168;201410169;201410170,
District7,41.00,24.00,14562857.74,8.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
District8,87.00,77.00,61689186.04,25.00,1.00,0.00,3852.83,-1383858.43,-359.18,1.00,0.00,3852.83,-1034136.93,-268.41,201201007,201201007
District9,60.00,42.00,33440295.38,9.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Total,802.00,552.00,476676706.29,169.00,52.00,32.00,4394334.82,99089790.55,22.55,42.00,24.00,3627268.33,44805428.36,12.35,0.00,0.00


In [92]:
# Crash Type for Districts
XLSPath = AnalysisDir
Index = list(Aggr_DF.index)[:-1]
HSIP_ID_L = Aggr_DF.HSIP_ID_OC
for i,ID in enumerate(Index):
    XLSList = [os.path.join(XLSPath, 'HSIP_' + f + '.xlsx') for f in HSIP_ID_L[i].split(';') if len(f)>6]  
    if len(XLSList)>0:
        df3 = pd.DataFrame()
        for xls in XLSList:
            p = int(os.path.basename(xls).split('_')[1].split('.')[0])
            df3 = pd.concat([df3,pd.read_excel(xls,'RawCrash')])
        PlotCrashType(df3,os.path.join(AnalysisDir,'Agg_' + str(ID) + '_CrashType.png'),ID)
        print(ID)    

District1
District2
District3
District4
District5
District6
District8


In [93]:
def HSIP_Aggr_StateLocal(HSIP_DF):
    Aggr_DF = pd.DataFrame()
    Index = ['State','Local']

    Aggr_DF['Tot']    = [len(HSIP_DF[HSIP_DF['Is Local Project']==i]) for i in [False,True]]

    Aggr_DF['W Tot Award']    = [len(HSIP_DF[[r['Is Local Project']==i and 
                                              r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Tot.Awrd.Amount']= [sum(HSIP_DF[[r['Is Local Project']==i and
                                              r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]['Total Award Amount']) for i in [False,True]]

    Aggr_DF['3Ys BA'] = [len(HSIP_DF[[r['Is Local Project']==i and
                         len(r['BeforePeriod'].split(';'))>3 and
                         len(r['AfterPeriod' ].split(';'))>3 and
                         r['Total Award Amount']>0
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_OC'] = [len(HSIP_DF[[r['Is Local Project']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_OC_BC>1'] = [len(HSIP_DF[[r['Is Local Project']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['BC_OC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['EUAC_OC'] = [sum(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in [False,True]]

    Aggr_DF['EUAB_OC'] = [sum(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_OC']) for i in [False,True]]

    Aggr_DF['BC_OC'] = [r['EUAB_OC']/(r['EUAC_OC'] or not r['EUAC_OC']) for i,r in Aggr_DF.iterrows()]

    Aggr_DF['Proc_EC'] = [len(HSIP_DF[[r['Is Local Project']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_EC_BC>1'] = [len(HSIP_DF[[r['Is Local Project']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']) and r['BC_EC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]
    
    Aggr_DF['EUAC_EC'] = [sum(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in [False,True]]
    Aggr_DF['EUAB_EC'] = [sum(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_EC']) for i in [False,True]]

    Aggr_DF['BC_EC'] = [r['EUAB_EC']/(r['EUAC_EC'] or not r['EUAC_EC']) for i,r in Aggr_DF.iterrows()]

    Aggr_DF['HSIP_ID_OC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in [False,True]]

    Aggr_DF['HSIP_ID_EC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[i==r['Is Local Project'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in [False,True]]

    Aggr_DF.index = Index
    #Aggr_DF['Total'] = [sum([r[c] for c in ['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA','Proc','EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC']]) for i,r in Aggr_DF.iterrows()]
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    return(Aggr_DF)
Aggr_DF = HSIP_Aggr_StateLocal(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
State,641.00,478.00,431788012.42,146.00,49.00,30.00,4264549.19,95532438.36,22.40,41.00,24.00,3579674.17,44830709.06,12.52,201112004;201201002;201201007;201203010;201309...,201112004;201201002;201201007;201203010;201309...
Local,161.00,74.00,44888693.87,23.00,3.00,2.00,129785.63,3557352.20,27.41,1.00,0.00,47594.17,-25280.70,-0.53,201410117;201410166;201410167,201410117
Total,802.00,552.00,476676706.29,169.00,52.00,32.00,4394334.82,99089790.55,22.55,42.00,24.00,3627268.33,44805428.36,12.35,0.00,0.00


In [94]:
# Crash Type for State/Local
XLSPath = AnalysisDir
Index = list(Aggr_DF.index)[:-1]
HSIP_ID_L = Aggr_DF.HSIP_ID_OC
for i,ID in enumerate(Index):
    XLSList = [os.path.join(XLSPath, 'HSIP_' + f + '.xlsx') for f in HSIP_ID_L[i].split(';') if len(f)>6]  
    if len(XLSList)>0:
        df3 = pd.DataFrame()
        for xls in XLSList:
            p = int(os.path.basename(xls).split('_')[1].split('.')[0])
            df3 = pd.concat([df3,pd.read_excel(xls,'RawCrash')])
        PlotCrashType(df3,os.path.join(AnalysisDir,'Agg_' + str(ID) + '_CrashType.png'),ID)
        print(ID)    

State
Local


In [99]:
def Extract_EAs(s,EA):
    try:
        if math.isnan(s):
            return([])
    except:
        s = str(s)
    L_List = s.split(' ')
    ea_List = {ea:False for ea in EA}
    for k in L_List:
        k = k.replace(',','')
        if len(k)<3:
            continue
        for ea in EA:
            if k in ea:
                ea_List[ea] = True
    return([ea for ea in ea_List if ea_List[ea]])
def HSIP_Aggr_EA(HSIP_DF):
    EA = ['Roadway Departures',
            'Intersections',
            'Driver Behavior & Awareness',
            'Large Trucks',
            'Highway-Railroad Grade Crossing',
            'Alcohol Related',
            'Information Systes',
            'Work Zones',
            'Safety Belts/Occupant Protection',
            'Vulnerable Users']

    Aggr_DF = pd.DataFrame(columns=['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA',
                                    'Proc_OC','Proc_OC_BC>1','EUAC_OC','EUAB_OC','BC_OC',
                                    'Proc_EC','Proc_EC_BC>1','EUAC_EC','EUAB_EC','BC_EC',
                                   'HSIP_ID_OC','HSIP_ID_EC'])
    Index = EA
    for c in Aggr_DF.columns:
        Aggr_DF[c] = [0.0 for ea in EA]
    Aggr_DF.index = Index
    Aggr_DF['HSIP_ID_OC'] = ''
    Aggr_DF['HSIP_ID_EC'] = ''
    
    for i,r in HSIP_DF.iterrows():
        ea_List = Extract_EAs(r['Emphasis Areas'],EA)
        for ea in ea_List:
            Aggr_DF.set_value(ea,'Tot',Aggr_DF.loc[ea]['Tot']+1) 

            if r['Total Award Amount']>0:
                Aggr_DF.set_value(ea,'W Tot Award',Aggr_DF.loc[ea]['W Tot Award']+1)
                Aggr_DF.set_value(ea,'Tot.Awrd.Amount',Aggr_DF.loc[ea]['Tot.Awrd.Amount']+r['Total Award Amount'])
            
                if len(r['BeforePeriod'].split(';'))>=3 and len(r['AfterPeriod'].split(';'))>=3:
                    Aggr_DF.set_value(ea,'3Ys BA',Aggr_DF.loc[ea]['3Ys BA']+1)
                    
                    if r['EUAC']>0 and r['EUAB_OC']!=0:
                        Aggr_DF.set_value(ea,'Proc_OC',Aggr_DF.loc[ea]['Proc_OC']+1)
                        Aggr_DF.set_value(ea,'EUAC_OC',Aggr_DF.loc[ea]['EUAC_OC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_OC',Aggr_DF.loc[ea]['EUAB_OC']+r['EUAB_OC'])
                        Aggr_DF.set_value(ea,'BC_OC'  ,Aggr_DF.loc[ea]['EUAB_OC']/Aggr_DF.loc[ea]['EUAC_OC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_OC',Aggr_DF.loc[ea]['HSIP_ID_OC']+';' + str(r['HSIP ID']))
                        if r['BC_OC']>1:
                            Aggr_DF.set_value(ea,'Proc_OC_BC>1',Aggr_DF.loc[ea]['Proc_OC_BC>1']+1)

                    if r['EUAC']>0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']):
                        Aggr_DF.set_value(ea,'Proc_EC',Aggr_DF.loc[ea]['Proc_EC']+1)
                        Aggr_DF.set_value(ea,'EUAC_EC',Aggr_DF.loc[ea]['EUAC_EC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_EC',Aggr_DF.loc[ea]['EUAB_EC']+r['EUAB_EC'])
                        Aggr_DF.set_value(ea,'BC_EC'  ,Aggr_DF.loc[ea]['EUAB_EC']/Aggr_DF.loc[ea]['EUAC_EC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_EC',Aggr_DF.loc[ea]['HSIP_ID_EC']+';' + str(r['HSIP ID']))
                        if r['BC_EC']>1:
                            Aggr_DF.set_value(ea,'Proc_EC_BC>1',Aggr_DF.loc[ea]['Proc_EC_BC>1']+1)
    for c in ['HSIP_ID_EC','HSIP_ID_OC']:
        Aggr_DF[c] = [s[1:] for s in Aggr_DF[c]]        
    Aggr_DF.index = Index
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    return(Aggr_DF)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
Aggr_DF = HSIP_Aggr_EA(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
Roadway Departures,344.00,254.00,247054782.90,129.00,20.00,14.00,1519234.98,70729203.57,46.56,16.00,8.00,1130279.78,39233524.26,34.71,201201007;201410002;201410003;201410004;201410...,201201007;201410002;201410003;201410004;201410...
Intersections,328.00,227.00,187485451.62,92.00,27.00,16.00,2611384.01,27169800.16,10.40,23.00,13.00,2324963.88,4795853.87,2.06,201201002;201203010;201410006;201410007;201410...,201201002;201203010;201410006;201410007;201410...
Driver Behavior & Awareness,43.00,16.00,4976930.85,5.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Large Trucks,11.00,5.00,6399158.87,1.00,1.00,0.00,174204.45,69467.14,0.40,1.00,1.00,174204.45,257420.38,1.48,201410044,201410044
Highway-Railroad Grade Crossing,1.00,1.00,36561.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Alcohol Related,3.00,1.00,660700.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Information Systes,2.00,2.00,2613359.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Work Zones,7.00,2.00,564781.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Safety Belts/Occupant Protection,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Vulnerable Users,30.00,17.00,10676021.17,9.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,


In [127]:
# Crash Type for State/Local
XLSPath = AnalysisDir
Index = list(Aggr_DF.index)[:-1]
HSIP_ID_L = Aggr_DF.HSIP_ID_OC
for i,ID in enumerate(Index):
    XLSList = [os.path.join(XLSPath, 'HSIP_' + f + '.xlsx') for f in HSIP_ID_L[i].split(';') if len(f)>6]  
    if len(XLSList)>0:
        df3 = pd.DataFrame()
        for xls in XLSList:
            p = int(os.path.basename(xls).split('_')[1].split('.')[0])
            df3 = pd.concat([df3,pd.read_excel(xls,'RawCrash')])
        PlotCrashType(df3,os.path.join(AnalysisDir,'Agg_' + str(ID) + '_CrashType.png'),ID)
        print(ID)    

3-D tape pavement marking
Add Left Turn Lane (ThreeLeg Intersection)
Add Right Turn Lane on One Approach
Add or Widen Paved Shoulder
Advance Warning with Flasher
Change Width of Existing Median (with Partial-No Access Control)
Chevrons or delineators
DeSlick (formerly known as skidproofing)
Delineators
Enhance and-or innovative traffic control devices (signs, signals and markings)
Enhance roadway lighting
Exclusive left-turn lanes
Fixed object removal and relocation
Guardrail
Impact Attenuators
Implement dynamic flashing beacons
Implement offset left turns at signalized intersections
Improve roadway geometrics and channelization
Improve sight distance at intersection approaches
Improvement-Realignment-Reconstruction
Install Advance Signal Warning Signs
Install Median Barrier (General)
Install Traffic Signals
Install milled-in rumble strips to both shoulders of Interstate roadways
Lane Addition
New median barrier devices and installations
Other Improvement
Raised Reflective Marker Media

In [101]:
def HSIP_Aggr_SystNonSyst(HSIP_DF):
    Aggr_DF = pd.DataFrame()
    Index = ['Non-Systemic','Systemic']

    Aggr_DF['Tot']    = [len(HSIP_DF[HSIP_DF['Systematic Improvements']==i]) for i in [False,True]]

    Aggr_DF['W Tot Award']    = [len(HSIP_DF[[r['Systematic Improvements']==i and 
                                              r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Tot.Awrd.Amount']= [sum(HSIP_DF[[r['Systematic Improvements']==i and
                                              r['Total Award Amount']>0 for j,r in HSIP_DF.iterrows()]]['Total Award Amount']) for i in [False,True]]

    Aggr_DF['3Ys BA'] = [len(HSIP_DF[[r['Systematic Improvements']==i and
                         len(r['BeforePeriod'].split(';'))>3 and
                         len(r['AfterPeriod' ].split(';'))>3 and
                         r['Total Award Amount']>0
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_OC'] = [len(HSIP_DF[[r['Systematic Improvements']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_OC_BC>1'] = [len(HSIP_DF[[r['Systematic Improvements']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['BC_OC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['EUAC_OC'] = [sum(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in [False,True]]

    Aggr_DF['EUAB_OC'] = [sum(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_OC']) for i in [False,True]]

    Aggr_DF['BC_OC'] = [r['EUAB_OC']/(r['EUAC_OC'] or not r['EUAC_OC']) for i,r in Aggr_DF.iterrows()]

    Aggr_DF['Proc_EC'] = [len(HSIP_DF[[r['Systematic Improvements']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]

    Aggr_DF['Proc_EC_BC>1'] = [len(HSIP_DF[[r['Systematic Improvements']==i and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']) and r['BC_EC']>1
                                      for j,r in HSIP_DF.iterrows()]]) for i in [False,True]]
    
    Aggr_DF['EUAC_EC'] = [sum(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAC']) for i in [False,True]]
    Aggr_DF['EUAB_EC'] = [sum(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                      for j,r in HSIP_DF.iterrows()]]['EUAB_EC']) for i in [False,True]]

    Aggr_DF['BC_EC'] = [r['EUAB_EC']/(r['EUAC_EC'] or not r['EUAC_EC']) for i,r in Aggr_DF.iterrows()]

    Aggr_DF['HSIP_ID_OC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in [False,True]]

    Aggr_DF['HSIP_ID_EC'] = [';'.join([str(ID) for ID in list(HSIP_DF[[i==r['Systematic Improvements'] and
                         len(r['BeforePeriod'].split(';'))>=3 and
                         len(r['AfterPeriod' ].split(';'))>=3 and
                         r['Total Award Amount']>0 and r['EUAC']!=0 and r['EUAB_OC']!=0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC'])
                                                                       for j,r in HSIP_DF.iterrows()]]['HSIP ID'])]) for i in [False,True]]

    Aggr_DF.index = Index
    #Aggr_DF['Total'] = [sum([r[c] for c in ['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA','Proc','EUAC','EUAB_OC','EUAB_EC','BC_OC','BC_EC']]) for i,r in Aggr_DF.iterrows()]
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    return(Aggr_DF)
Aggr_DF = HSIP_Aggr_SystNonSyst(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
Non-Systemic,551.00,370.00,330530545.39,103.00,41.00,22.00,3752293.13,60950933.95,16.24,32.00,18.00,3025353.44,36761694.97,12.15,201112004;201201002;201201007;201203010;201309...,201112004;201201002;201201007;201203010;201309...
Systemic,171.00,134.00,99299862.55,44.00,11.00,10.00,642041.69,38138856.61,59.40,10.00,6.00,601914.90,8043733.40,13.36,201410003;201410014;201410017;201410075;201410...,201410003;201410014;201410017;201410075;201410...
Total,722.00,504.00,429830407.94,147.00,52.00,32.00,4394334.82,99089790.55,22.55,42.00,24.00,3627268.33,44805428.36,12.35,0.00,0.00


In [126]:
def ExtractImps(s):
    s = s.replace(u'\u201c','')
    s = s.replace(u'\u201d','')
    s = s.replace(u'/','-')
    s = s.replace(u'>','to')
    l = s.split(';')
    l = [''.join(k.split('.')) for k in l if len(k)>0]
    l2 = []
    for k in l:
        if 'Other Improvement' in k:
            l2.append(u'Other Improvement')
            l2.append(k.replace('Other Improvement',''))
        else:
            l2.append(k)
    return([k for k in l2 if len(k)>0])
def HSIP_Aggr_Imp(HSIP_DF):
    Imps = []
    for s in list(HSIP_DF['All Selected Improvements']):
        Imps.extend(ExtractImps(s))
    Imps = list(set(Imps))
    Imps.sort()

    Aggr_DF = pd.DataFrame(columns=['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA',
                                    'Proc_OC','Proc_OC_BC>1','EUAC_OC','EUAB_OC','BC_OC',
                                    'Proc_EC','Proc_EC_BC>1','EUAC_EC','EUAB_EC','BC_EC',
                                   'HSIP_ID_OC','HSIP_ID_EC'])
    Index = Imps
    for c in Aggr_DF.columns:
        Aggr_DF[c] = [0.0 for ea in Imps]
    Aggr_DF.index = Index
    Aggr_DF['HSIP_ID_OC'] = ''
    Aggr_DF['HSIP_ID_EC'] = ''
    
    for i,r in HSIP_DF.iterrows():
        ea_List = ExtractImps(r['All Selected Improvements'])
        for ea in ea_List:
            Aggr_DF.set_value(ea,'Tot',Aggr_DF.loc[ea]['Tot']+1) 

            if r['Total Award Amount']>0:
                Aggr_DF.set_value(ea,'W Tot Award',Aggr_DF.loc[ea]['W Tot Award']+1)
                Aggr_DF.set_value(ea,'Tot.Awrd.Amount',Aggr_DF.loc[ea]['Tot.Awrd.Amount']+r['Total Award Amount'])
            
                if len(r['BeforePeriod'].split(';'))>=3 and len(r['AfterPeriod'].split(';'))>=3:
                    Aggr_DF.set_value(ea,'3Ys BA',Aggr_DF.loc[ea]['3Ys BA']+1)
                    
                    if r['EUAC']>0 and r['EUAB_OC']!=0:
                        Aggr_DF.set_value(ea,'Proc_OC',Aggr_DF.loc[ea]['Proc_OC']+1)
                        Aggr_DF.set_value(ea,'EUAC_OC',Aggr_DF.loc[ea]['EUAC_OC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_OC',Aggr_DF.loc[ea]['EUAB_OC']+r['EUAB_OC'])
                        Aggr_DF.set_value(ea,'BC_OC'  ,Aggr_DF.loc[ea]['EUAB_OC']/Aggr_DF.loc[ea]['EUAC_OC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_OC',Aggr_DF.loc[ea]['HSIP_ID_OC']+';' + str(r['HSIP ID']))
                        if r['BC_OC']>1:
                            Aggr_DF.set_value(ea,'Proc_OC_BC>1',Aggr_DF.loc[ea]['Proc_OC_BC>1']+1)

                    if r['EUAC']>0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']):
                        Aggr_DF.set_value(ea,'Proc_EC',Aggr_DF.loc[ea]['Proc_EC']+1)
                        Aggr_DF.set_value(ea,'EUAC_EC',Aggr_DF.loc[ea]['EUAC_EC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_EC',Aggr_DF.loc[ea]['EUAB_EC']+r['EUAB_EC'])
                        Aggr_DF.set_value(ea,'BC_EC'  ,Aggr_DF.loc[ea]['EUAB_EC']/Aggr_DF.loc[ea]['EUAC_EC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_EC',Aggr_DF.loc[ea]['HSIP_ID_EC']+';' + str(r['HSIP ID']))
                        if r['BC_EC']>1:
                            Aggr_DF.set_value(ea,'Proc_EC_BC>1',Aggr_DF.loc[ea]['Proc_EC_BC>1']+1)
    for c in ['HSIP_ID_EC','HSIP_ID_OC']:
        Aggr_DF[c] = [s[1:] for s in Aggr_DF[c]]        
    Aggr_DF.index = Index
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    return(Aggr_DF)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
Aggr_DF = HSIP_Aggr_Imp(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
3-D tape pavement marking,1.00,1.00,707787.45,1.00,1.00,1.00,55083.89,1485403.05,26.97,1.00,1.00,55083.89,109955.53,2.00,201410015,201410015
Add Lanes by Narrowing Existing Lanes,1.00,1.00,931829.32,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add Left Turn Lane (ThreeLeg Intersection),10.00,8.00,11763930.16,5.00,1.00,0.00,108093.67,-622850.24,-5.76,1.00,0.00,108093.67,-349048.76,-3.23,201203010,201203010
Add Left Turn Phase on One Approach (Permissive or Perm-Prot to Protected),12.00,9.00,15626240.96,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add Left Turn Phase on One Approach (Permissive to Perm-Prot),10.00,7.00,7315859.32,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add Mast Arms and Signal Head per Lane,70.00,41.00,30258485.45,6.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add Pedestrian Controls,7.00,3.00,2101481.60,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add Right Turn Lane on One Approach,16.00,9.00,7882492.40,3.00,1.00,0.00,108093.67,-622850.24,-5.76,1.00,0.00,108093.67,-349048.76,-3.23,201203010,201203010
Add Traffic Actuation,1.00,1.00,1187745.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,,
Add or Widen Paved Shoulder,59.00,41.00,43509009.63,25.00,5.00,4.00,388373.90,11406319.65,29.37,4.00,2.00,336167.08,1693768.24,5.04,201410016;201410017;201410112;201410115;201410116,201410016;201410017;201410112;201410116


In [137]:
def CostCat(a):
    if a<=100000:
        return('Low')
    if a<=1000000:
        return('Med')
    return('High')
def HSIP_Aggr_Cost(HSIP_DF):
    Aggr_DF = pd.DataFrame(columns=['Tot','W Tot Award','Tot.Awrd.Amount','3Ys BA',
                                    'Proc_OC','Proc_OC_BC>1','EUAC_OC','EUAB_OC','BC_OC',
                                    'Proc_EC','Proc_EC_BC>1','EUAC_EC','EUAB_EC','BC_EC',
                                   'HSIP_ID_OC','HSIP_ID_EC'])
    Index = ['Low','Med','High']
    for c in Aggr_DF.columns:
        Aggr_DF[c] = [0.0 for ea in Index]
    Aggr_DF.index = Index
    Aggr_DF['HSIP_ID_OC'] = ''
    Aggr_DF['HSIP_ID_EC'] = ''
    
    for i,r in HSIP_DF.iterrows():
        ea_List = [CostCat(r['Total Award Amount'])]
        for ea in ea_List:
            Aggr_DF.set_value(ea,'Tot',Aggr_DF.loc[ea]['Tot']+1) 

            if r['Total Award Amount']>0:
                Aggr_DF.set_value(ea,'W Tot Award',Aggr_DF.loc[ea]['W Tot Award']+1)
                Aggr_DF.set_value(ea,'Tot.Awrd.Amount',Aggr_DF.loc[ea]['Tot.Awrd.Amount']+r['Total Award Amount'])
            
                if len(r['BeforePeriod'].split(';'))>=3 and len(r['AfterPeriod'].split(';'))>=3:
                    Aggr_DF.set_value(ea,'3Ys BA',Aggr_DF.loc[ea]['3Ys BA']+1)
                    
                    if r['EUAC']>0 and r['EUAB_OC']!=0:
                        Aggr_DF.set_value(ea,'Proc_OC',Aggr_DF.loc[ea]['Proc_OC']+1)
                        Aggr_DF.set_value(ea,'EUAC_OC',Aggr_DF.loc[ea]['EUAC_OC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_OC',Aggr_DF.loc[ea]['EUAB_OC']+r['EUAB_OC'])
                        Aggr_DF.set_value(ea,'BC_OC'  ,Aggr_DF.loc[ea]['EUAB_OC']/Aggr_DF.loc[ea]['EUAC_OC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_OC',Aggr_DF.loc[ea]['HSIP_ID_OC']+';' + str(r['HSIP ID']))
                        if r['BC_OC']>1:
                            Aggr_DF.set_value(ea,'Proc_OC_BC>1',Aggr_DF.loc[ea]['Proc_OC_BC>1']+1)

                    if r['EUAC']>0 and r['EUAB_EC']!=0 and not math.isnan(r['EUAB_EC']):
                        Aggr_DF.set_value(ea,'Proc_EC',Aggr_DF.loc[ea]['Proc_EC']+1)
                        Aggr_DF.set_value(ea,'EUAC_EC',Aggr_DF.loc[ea]['EUAC_EC']+r['EUAC'])
                        Aggr_DF.set_value(ea,'EUAB_EC',Aggr_DF.loc[ea]['EUAB_EC']+r['EUAB_EC'])
                        Aggr_DF.set_value(ea,'BC_EC'  ,Aggr_DF.loc[ea]['EUAB_EC']/Aggr_DF.loc[ea]['EUAC_EC'])
                        Aggr_DF.set_value(ea,'HSIP_ID_EC',Aggr_DF.loc[ea]['HSIP_ID_EC']+';' + str(r['HSIP ID']))
                        if r['BC_EC']>1:
                            Aggr_DF.set_value(ea,'Proc_EC_BC>1',Aggr_DF.loc[ea]['Proc_EC_BC>1']+1)
    for c in ['HSIP_ID_EC','HSIP_ID_OC']:
        Aggr_DF[c] = [s[1:] for s in Aggr_DF[c]]        
    Aggr_DF.index = Index
    TotL = []
    for c in Aggr_DF.columns:
        if c in ['HSIP_ID_EC','HSIP_ID_OC']:
            TotL.append(0)
        else:
            TotL.append(sum(Aggr_DF[c]))
    Aggr_DF.loc['Total'] = TotL
    Aggr_DF.set_value('Total','BC_OC',Aggr_DF.loc['Total']['EUAB_OC']/Aggr_DF.loc['Total']['EUAC_OC'])
    Aggr_DF.set_value('Total','BC_EC',Aggr_DF.loc['Total']['EUAB_EC']/Aggr_DF.loc['Total']['EUAC_EC'])
    return(Aggr_DF)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
Aggr_DF = HSIP_Aggr_Cost(HSIP_DF)
display(Aggr_DF)

,Tot,W Tot Award,Tot.Awrd.Amount,3Ys BA,Proc_OC,Proc_OC_BC>1,EUAC_OC,EUAB_OC,BC_OC,Proc_EC,Proc_EC_BC>1,EUAC_EC,EUAB_EC,BC_EC,HSIP_ID_OC,HSIP_ID_EC
Low,335.00,85.00,3979785.61,34.00,5.00,2.00,22140.61,-1178778.46,-53.24,5.00,2.00,22140.61,-954056.56,-43.09,201112004;201201007;201410033;201410077;201410079,201112004;201201007;201410033;201410077;201410079
Med,314.00,314.00,143045136.43,157.00,28.00,21.00,1205394.76,83314500.47,69.12,22.00,14.00,928600.78,43934329.48,47.31,201201002;201309010;201410002;201410003;201410...,201201002;201309010;201410002;201410003;201410...
High,153.00,153.00,329651784.25,67.00,19.00,9.00,3166799.45,16954068.55,5.35,16.00,8.00,2717598.18,1689423.07,0.62,201203010;201410005;201410010;201410016;201410...,201203010;201410005;201410016;201410020;201410...
Total,802.00,552.00,476676706.29,258.00,52.00,32.00,4394334.82,99089790.55,22.55,43.00,24.00,3668339.57,44669695.99,12.18,0.00,0.00
